In [13]:
import numpy as np
import time
import soundfile as sf

In [14]:
tube_length = 36
fs = 44100
embochure = 20
inputGain = 0.372
duration = fs*3

In [15]:
def noise(duration):
    y = (np.random.rand(duration,1)-0.5)*2
    return y

def normalize(array):
    min = np.min(array)
    max = np.max(array)
    normalized_array = 2*(array-min)/(max-min) - 1
    return normalized_array


In [16]:
# Valimaki 1992

# Noise ノイズ
burst_noise = noise(duration)

# Variables 変数
output = np.empty(duration)  # Preallocate output array for efficiency
delayTube1 = np.empty(duration)
delayTube2 = np.empty(duration)
delayEmb = np.empty(duration)
delayFb = np.empty(duration)
delayFf = np.empty(duration)

output1 = 0
output2 = 0
feedback1 = 0
feedback2 = 0

delayfr = 0

start = 0
end = 0

start = time.time()
for i in range(duration):
    instantSample = burst_noise[i]

    # + Feedback
    instantSample += feedback1

    # Input Gain
    instantSample *= inputGain

    # Delay-line Emb
    delayEmb[i] = instantSample
    if i > embochure:
        instantSample = delayEmb[i - embochure]

    # Nonlinearity
    instantSample = instantSample - (instantSample ** 3)

    # + Feedback
    instantSample += feedback2

    # Output1
    output1 = instantSample

    # Delay-line 1
    delayTube1[i] = instantSample
    if i > tube_length:
        instantSample = delayTube1[i - tube_length]

    # Fr
    instantSample_to_fr = instantSample
    instantSample_to_fr = instantSample * 0.11 + delayfr * (-0.89)
    delayfr = instantSample_to_fr
    output2 = instantSample_to_fr

    # Ff
    instantSample_to_ff = instantSample
    delayFf[i] = instantSample_to_ff
    if i > 3:
        instantSample_to_ff = instantSample_to_ff * 0.25 + delayFf[i-1] * 0.25 + delayFf[i-2] * 0.25 + delayFf[i-3] * 0.25
    instantSample = instantSample_to_ff

    # Delay-line 2
    delayTube2[i] = instantSample
    if i > tube_length:
        instantSample = delayTube2[i - tube_length]
    instantSample_start = instantSample
    
    # Fb
    instantSample_to_fb = instantSample
    delayFb[i] = instantSample_to_fb
    if i > 3:
        instantSample_to_fb = instantSample_to_fb * 0.25 + delayFb[i-1] * 0.25 + delayFb[i-2] * 0.25 + delayFb[i-3] * 0.25
    
    # feedback1
    feedback1 = instantSample_start * 0.1

    # feedback2
    feedback2 = instantSample_to_fb

    # Output
    output[i] = (output1 + output2) * 0.03162277660168379 # -30dB

normalized_output = normalize(output)
print(normalized_output)
end = time.time()

[-0.79449074 -0.78516793 -0.77310287 ...  0.95005501  0.95933628
  0.93870812]


In [17]:
# write output to file for listening
sf.write('output_Valimaki92.wav', normalized_output, fs)

elapsed_time = end - start
print(elapsed_time)

1.3975818157196045
